In [1]:

import pandas as pd
import scipy
import pyodbc

import time

conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=LAPTOP-IFK6D8L3\\SQLEXPRESS;DATABASE=StockQuote;UID=sa;PWD=password')
cur = conn.cursor()


SQL = """ select Script_Name, Trnx_date trnx, [close]cls from NSE_EOD where Trnx_date in(select distinct top 300 Trnx_date from NSE_EOD order by Trnx_date desc )
"""

df = pd.read_sql(SQL,conn)
display(df)

,Script_Name,trnx,cls
0,20MICRONS,2021-11-08,60.05
1,20MICRONS,2021-11-09,61.65
2,20MICRONS,2021-11-10,62.75
3,20MICRONS,2021-11-11,66.10
4,20MICRONS,2021-11-12,64.45
...,...,...,...
532602,ZYDUSWELL,2022-12-19,1587.35
532603,ZYDUSWELL,2022-12-20,1588.85
532604,ZYDUSWELL,2022-12-21,1571.95
532605,ZYDUSWELL,2022-12-22,1542.50


In [2]:
df_gby = (
        (df
            .groupby(['Script_Name', 'trnx'])['cls']
            .mean()
        )
        .ewm(span=12)
        .mean()
        # .droplevel(2)
        # .reset_index(name='ema'+str(days))
    )

df_gby

Script_Name  trnx      
20MICRONS    2021-11-08      60.050000
             2021-11-09      60.916667
             2021-11-10      61.632217
             2021-11-11      63.042522
             2021-11-12      63.424928
                              ...     
ZYDUSWELL    2022-12-19    1600.728625
             2022-12-20    1598.901144
             2022-12-21    1594.754814
             2022-12-22    1586.715612
             2022-12-23    1572.382441
Name: cls, Length: 532607, dtype: float64

In [10]:
def ewm(df, days):
    df_gby = (
        (df
            .groupby(['Script_Name', 'trnx'])['cls']
            .sum()
        )
        .ewm(span=days, min_periods=days)
        .mean()
        .droplevel(1)
        .reset_index(name='ema'+str(days))
    )
    
    return df_gby['ema'+str(days)]

In [11]:
for i in [9,12,21,25,26,50,100]:
    df['ema'+str(i)] = ewm(df, i)

df

,Script_Name,trnx,cls,ema9,ema12,ema21,ema25,ema26,ema50,ema100
0,20MICRONS,2021-11-08,60.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20MICRONS,2021-11-09,61.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20MICRONS,2021-11-10,62.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20MICRONS,2021-11-11,66.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20MICRONS,2021-11-12,64.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
532602,ZYDUSWELL,2022-12-19,1587.35,1599.946681,1600.728625,1606.611123,1610.170317,1611.058337,1625.763973,1626.280924
532603,ZYDUSWELL,2022-12-20,1588.85,1597.727345,1598.901144,1604.996476,1608.530292,1609.413275,1624.316366,1625.539717
532604,ZYDUSWELL,2022-12-21,1571.95,1592.571876,1594.754814,1601.992251,1605.716424,1606.638218,1622.262783,1624.478535
532605,ZYDUSWELL,2022-12-22,1542.50,1582.557501,1586.715612,1596.583864,1600.853622,1601.887238,1619.134831,1622.855198


In [9]:
df.to_csv("./test1.csv")